In [ ]:
import json
import pandas as pd
import numpy as np
import re
import string
import nltk
from num2words import num2words

In [1]:
## Read the data into a dataframe

df = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/textdataset/WELFake/WELFake_Dataset.csv')

df = df.dropna()
df.isnull().sum()

df.drop(columns=['Unnamed: 0'],inplace=True)

df.drop_duplicates(inplace=True)

X = df.drop(columns=['label'])
y = df['label']

In [2]:
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

X[['title','text']] = X[['title','text']].applymap(lambda x:remove_punctuation(x))
X[['title','text']] = X[['title','text']].applymap(lambda x:x.lower())

In [3]:
def clean_text(text):
    text=str(text).lower() #Converts text to lowercase
    text=re.sub('\d+', '', text) #removes numbers
    text=re.sub('\[.*?\]', '', text) #removes HTML tags
    text=re.sub('https?://\S+|www\.\S+', '', text) #removes url
    text=re.sub(r"["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", "", text) #removes emojis
    text=re.sub('[%s]' % re.escape(string.punctuation),'',text) #removes punctuations
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [4]:
X=X['title']+' '+X['text']

X = X.apply(clean_text)

In [5]:
X = list(X)

pattern = '[^a-z ]'
Clean_X=[]
for sen in X:
    Clean_X.append(re.sub(pattern, '', str(sen)))

In [6]:
clean_df = pd.DataFrame({'Clean_X': Clean_X, 'y': y})

In [7]:
fake_df = clean_df[clean_df['y'] == 0]
real_df = clean_df[clean_df['y'] == 1]

In [8]:
fake_x=list(fake_df['Clean_X'])
real_x=list(real_df['Clean_X'])

In [9]:
real_selected_lst = []
fake_selected_lst = []

for sen in real_x:
    word_count = len(sen.split())
    if 10 <= word_count < 2000:
        real_selected_lst.append(sen)
        
for sen in fake_x:
    word_count = len(sen.split())
    if 10 <= word_count < 2000:
        fake_selected_lst.append(sen)

In [10]:
X=real_selected_lst[:10000]+fake_selected_lst[:10000]
y=[0]*10000+[1]*10000

In [11]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
to_txt=x_train+x_test

In [14]:
to_txt

['lol cnn interview abruptly ends when student who started protest at betsy devos speech realizes he cant explain why hes protesting video the students of the historically black bethunecookman university booed the invited commencement speaker education secretary betsy devos they yelled over her speech they even turned their backs on her for  minutes the students at bethune cookman reminded us just how far obama has divided our nationa sophomore student at bethunecookman a historically black university filmed himself running around with his phone as he yelled for the seniors to stand and rebuke betsy devos during her commencement speech he implored the graduating seniors to disrupt her speech by yelling and standing with their backs to  devos who bravely persevered through her speech in spite of their embarrassing behaviorthe student who says his name is  bobbie luke  and identifies himself as a  stupid nigga  on his twitter account was eventually thrown out of the venue by campus secur

In [13]:
len(to_txt)

20000

In [15]:
import pickle

with open("WELFake_text.pkl","wb") as f:
    pickle.dump(to_txt, f)

In [16]:
#문장 끝에 온점 추가
for i in range(len(to_txt)):
    to_txt[i]=to_txt[i] + " ."

In [17]:
import numpy as np

train_data=np.array(to_txt)

In [18]:
import os
if not os.path.exists("swb/train"):
    os.makedirs("swb/train")
for i in range(0,train_data.shape[0],6):
    text = "\n".join(train_data[i:i+6].tolist())
    fp = open("swb/train/"+str(i)+".txt","w")
    fp.write(text)
    fp.close()

In [19]:
#소문자
from collections import Counter

texts = " ".join(train_data.tolist())
words = texts.split(" ")
print("Number of tokens in Training data = ",len(words))
dictionary = Counter(words)
print("Size of Vocab",len(dictionary))
sorted_vocab = ["<S>","</S>","<UNK>"]
sorted_vocab.extend([pair[0] for pair in dictionary.most_common()])
 
text = "\n".join(sorted_vocab)
fp = open("swb/vocab.txt","w")
fp.write(text)
fp.close()

Number of tokens in Training data =  10289853
Size of Vocab 177566


In [20]:
#하이퍼 파라미터 옵션(학습이 끝난 뒤엔 꼭 n_character=262로 해줘야 하므로 재설정. 기본값이 261로 돼있을 것임)
options = {
     'bidirectional': True,

     'char_cnn': {'activation': 'relu',
      'embedding': {'dim': 16},
      'filters': [[1, 32],
       [2, 32],
       [3, 64],
       [4, 128],
       [5, 256],
       [6, 512],
       [7, 1024]],
      'max_characters_per_token': 50,
      'n_characters': 262,
      'n_highway': 2},
    
     'dropout': 0.1,
    
     'lstm': {
      'cell_clip': 3,
      'dim': 1024,    #출력 임베딩 벡터의 4배로 설정
      'n_layers': 2,
      'proj_clip': 3,
      'projection_dim': 128,    #출력 임베딩 벡터의 1/2로 설정 예) ELMo 임베딩 벡터의 출력이 256이라면 128로 설정
      'use_skip_connections': True},
    
     'all_clip_norm_val': 10.0,
    
     'n_epochs': 10,
     'n_train_tokens': 10289853,    #위 코드에서 찾은 Number of tokens in Training data
     'batch_size': 64,
     'n_tokens_vocab': 177566,   #위 코드에서 찾은 Size of Vocab
     'unroll_steps': 20,
     'n_negative_samples_batch': 8192,
    }

In [21]:
import json
file_path = "./swb/checkpoint/options.json"
with open(file_path, 'w') as outfile:
    json.dump(options, outfile, indent=4)